## Advanced Box Score Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a game's play by play data and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [28]:
import pandas as pd

import requests
import pandas as pd
import json
import html
import os.path

selected_team = 'Georgia Tech'
selected_opponent = 'Virginia'
selected_week = 11
selected_team_is_home = False

def retrieveCfbData(endpoint, team, week):
    file_path = f"data/{endpoint if (endpoint != 'plays') else 'pbp'}/{endpoint[:-1] if (endpoint != 'plays') else 'pbp'}-data-{team.lower().replace(' ','-')}-wk{week}.json"
    if (os.path.exists(file_path)):
        return file_path
    res = requests.get(f"https://api.collegefootballdata.com/{endpoint}?seasonType=regular&year=2019&team={html.escape(team)}&week={week}")
    content = res.json()
    with open(file_path, 'w') as f:
        json.dump(content, f)
    return json.dumps(content)
    

pbp_data = pd.read_json(retrieveCfbData('plays',selected_team, selected_week))
# pbp_data.info()

drives = pd.read_json(retrieveCfbData('drives',selected_team, selected_week))
drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','game_id'], axis = 1, inplace=True) 
drives = drives[
    (~drives.drive_result.isin(['END OF HALF','END OF GAME','Uncategorized']))
    & (~drives.drive_result.str.contains('END OF'))
]

away_team = selected_team if ~selected_team_is_home else selected_opponent
home_team = selected_opponent if ~selected_team_is_home else selected_team

In [29]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives.loc[
    drives.offense == away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))

pbp_data = pbp_data[
    (pbp_data.down != 0)
]

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]


# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

print("Total clean plays:", len(pbp_data))

Total Drives: 20
Total Plays: 127
Garbage Time Plays: 0
Non-Garbage Time Plays: 127
Total clean plays: 127


In [30]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

pbp_data.loc[
    ((pbp_data.play_type.isin(bad_types))
     & (~pbp_data.play_type.str.contains('Sack'))) ,['yards_gained']] = 0

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    elif ((down == 4) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)

In [31]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

print("Success Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

print("")
print("Success Rate by Q")
for i in range(1, pbp_data.period.nunique()+1):
    print(f"Q{i}\n{away_team}:", len(pbp_data[(pbp_data.period == i) & (pbp_data.offense == away_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.period == i) & (pbp_data.offense == away_team)]),f"\n{home_team}:", len(pbp_data[(pbp_data.period == i) & (pbp_data.offense == home_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.period == i) & (pbp_data.offense == home_team)]))

Success Rates
Georgia Tech
Overall: 0.42592592592592593
Passing: 0.5454545454545454
Rushing: 0.34375
Virginia
Overall: 0.4383561643835616
Passing: 0.4444444444444444
Rushing: 0.43243243243243246

Success Rate by Q
Q1
Georgia Tech: 0.4375 
Virginia: 0.5
Q2
Georgia Tech: 0.4666666666666667 
Virginia: 0.38095238095238093
Q3
Georgia Tech: 0.25 
Virginia: 0.42105263157894735
Q4
Georgia Tech: 0.4666666666666667 
Virginia: 0.47058823529411764


In [32]:
###### Explosiveness rates
# Defined as rate of 15+ yards gains

print("Explosiveness Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))

Explosiveness Rates
Georgia Tech
Overall: 0.12962962962962962
Passing: 0.18181818181818182
Rushing: 0.09375
Virginia
Overall: 0.0958904109589041
Passing: 0.16666666666666666
Rushing: 0.02702702702702703


In [33]:
# Havoc plays
havoc_plays = pbp_data[
    (((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (pbp_data.play_type == 'Sack')
    | (pbp_data.play_type.str.contains('Interception', regex=False))
    | (pbp_data.yards_gained < 0))
    & (pbp_data.play_type != 'Penalty')
]

print(away_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == away_team
]), "/", len(pbp_data[
    pbp_data.defense == away_team
]), "(",len(havoc_plays[
    havoc_plays.defense == away_team
]) / len(pbp_data[
    pbp_data.defense == away_team
]),")")

print(home_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == home_team
]), "/", len(pbp_data[
    pbp_data.defense == home_team
]), "(",len(havoc_plays[
    havoc_plays.defense == home_team
]) / len(pbp_data[
    pbp_data.defense == home_team
]),")")

Georgia Tech Havoc Rate:  12 / 73 ( 0.1643835616438356 )
Virginia Havoc Rate:  4 / 54 ( 0.07407407407407407 )


In [34]:
# Calculating converted 3rd and 8+s
ignore_turnovers = ['Fumble Recovery (Opponent)','Fumble Return Touchdown',"Interception","Pass Interception Return","Interception Return Touchdown"]

third_longs = pbp_data[
    (pbp_data.down == 3)
    & (pbp_data.distance >= 8)
    & (~pbp_data.play_type.isin(ignore_turnovers))
]
# converted_3rd_longs

away_team_thirds = third_longs[
    (third_longs.offense == away_team)
]

away_team_thirds_converted = away_team_thirds[
    (away_team_thirds.yards_gained >= 8)
]
print("Converted 3rd and 8+ for", away_team,":",len(away_team_thirds_converted),"of",len(away_team_thirds))

home_team_thirds = third_longs[
    (third_longs.offense == home_team)
]

home_team_thirds_converted = home_team_thirds[
    (home_team_thirds.yards_gained >= 8)
]

print("Converted 3rd and 8+ for", home_team,":",len(home_team_thirds_converted),"of",len(home_team_thirds))

Converted 3rd and 8+ for Georgia Tech : 0 of 4
Converted 3rd and 8+ for Virginia : 2 of 5


In [35]:
away_team_drives = drives[
    drives.offense == away_team
]

print(away_team, "Drives:", len(away_team_drives))
print(away_team,"Yards:",sum(away_team_drives.yards))
print(away_team,"Plays:",sum(away_team_drives.plays))
print(away_team, "Avg Starting Field Position:", sum(away_team_drives.start_yardline) / len(away_team_drives))
print(away_team, "Yards per Play:", sum(away_team_drives.yards) / sum(away_team_drives.plays))
print(away_team, "Plays per Drive:", sum(away_team_drives.plays) / len(away_team_drives))
print(away_team, "Yards per Drive:", sum(away_team_drives.yards) / len(away_team_drives))
#away_team_drives
print("")
home_team_drives = drives[
    drives.offense == home_team
]

print(home_team, "Drives:", len(home_team_drives))
print(home_team,"Yards:",sum(home_team_drives.yards))
print(home_team,"Plays:",sum(home_team_drives.plays))
print(home_team, "Avg Starting Field Position:", sum(home_team_drives.start_yardline) / len(home_team_drives))
print(home_team, "Yards per Play:", sum(home_team_drives.yards) / sum(home_team_drives.plays))
print(home_team, "Plays per Drive:", sum(home_team_drives.plays) / len(home_team_drives))
print(home_team, "Yards per Drive:", sum(home_team_drives.yards) / len(home_team_drives))
#home_team_drives

Georgia Tech Drives: 10
Georgia Tech Yards: 403
Georgia Tech Plays: 54
Georgia Tech Avg Starting Field Position: 22.1
Georgia Tech Yards per Play: 7.462962962962963
Georgia Tech Plays per Drive: 5.4
Georgia Tech Yards per Drive: 40.3

Virginia Drives: 10
Virginia Yards: 405
Virginia Plays: 65
Virginia Avg Starting Field Position: 38.4
Virginia Yards per Play: 6.230769230769231
Virginia Plays per Drive: 6.5
Virginia Yards per Drive: 40.5


In [36]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [37]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = pbp_data[
    pbp_data.play_type.isin(pass_types)
]
rush_plays = pbp_data[
    pbp_data.play_type.isin(rush_types)
]

In [38]:
print(away_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == away_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == away_team)
]))

print(away_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == away_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == away_team)
]))

print(home_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == home_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == home_team)
]))

print(home_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == home_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == home_team)
]))

Georgia Tech Success Rate on Std Downs: 0.4857142857142857
Georgia Tech Success Rate on Passing Downs: 0.3157894736842105
Virginia Success Rate on Std Downs: 0.5087719298245614
Virginia Success Rate on Passing Downs: 0.1875


In [39]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

away_team_stuffs = stuffed_plays[
    stuffed_plays.defense == away_team
]
print(away_team,"Defensive Stuff Rate: ",len(away_team_stuffs),"/",len(rush_plays[rush_plays.defense == away_team]),"(",len(away_team_stuffs)/len(rush_plays[rush_plays.defense == away_team]),")")

home_team_stuffs = stuffed_plays[
    stuffed_plays.defense == home_team
]
print(home_team,"Defensive Stuff Rate: ",len(home_team_stuffs),"/",len(rush_plays[rush_plays.defense == home_team]),"(",len(home_team_stuffs)/len(rush_plays[rush_plays.defense == home_team]),")")

Georgia Tech Defensive Stuff Rate:  4 / 37 ( 0.10810810810810811 )
Virginia Defensive Stuff Rate:  5 / 32 ( 0.15625 )


In [40]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 5)
]
print(away_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == away_team]),"/",len(rush_plays[rush_plays.offense == away_team]),"(",len(rush_opps[rush_opps.offense == away_team])/len(rush_plays[rush_plays.offense == away_team]),")")

print(home_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == home_team]),"/",len(rush_plays[rush_plays.offense == home_team]),"(",len(rush_opps[rush_opps.offense == home_team])/len(rush_plays[rush_plays.offense == home_team]),")")

Georgia Tech Rush Opp Rate:  11 / 32 ( 0.34375 )
Virginia Rush Opp Rate:  9 / 37 ( 0.24324324324324326 )


In [41]:
##### Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif ((yards_gained >= 0) & (yards_gained <= 3)):
        return yards_gained * 1.0
    elif ((yards_gained >= 4) & (yards_gained <= 6)):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

rush_opps['line_yards'] = rush_opps.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_opps['highlight_yards'] = rush_opps.apply(lambda x: x.yards_gained - x.line_yards, axis=1)

print(away_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].line_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].highlight_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].yards_gained) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Hlt Yards per Opp:",sum(rush_opps[rush_opps.offense == away_team].highlight_yards) / len(rush_opps[rush_opps.offense == away_team]))

print("")
print(home_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].line_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].highlight_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].yards_gained) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Hlt Yards per Opp:",sum(rush_opps[rush_opps.offense == home_team].highlight_yards) / len(rush_opps[rush_opps.offense == home_team]))

rush_plays.head()

Georgia Tech Line Yards per Carry: 0.8046875
Georgia Tech Highlight Yards per Carry: 4.1328125
Georgia Tech Yards per Carry: 4.9375
Georgia Tech Hlt Yards per Opp: 10.863636363636363

Virginia Line Yards per Carry: 0.9932432432432432
Virginia Highlight Yards per Carry: 3.195945945945946
Virginia Yards per Carry: 4.1891891891891895
Virginia Hlt Yards per Opp: 11.11111111111111


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,away,defense,defense_score,distance,down,drive_id,home,id,offense,offense_score,period,play_text,play_type,yard_line,yards_gained,play_successful,play_explosive,line_yards,highlight_yards
2,Georgia Tech,Virginia,0,7.0,2,4011125041,Virginia,401112504101849920,Georgia Tech,0,1,Jordan Mason run for 4 yds to the GTech 32,Rush,72,4,False,False,2.0,2.0
3,Georgia Tech,Virginia,0,10.0,1,4011125041,Virginia,401112504101849920,Georgia Tech,0,1,James Graham run for 3 yds to the GTech 28,Rush,75,3,False,False,3.0,0.0
5,Georgia Tech,Georgia Tech,7,9.0,2,4011125042,Virginia,401112504101895808,Virginia,0,1,Wayne Taulapapa run for 5 yds to the GTech 4,Rush,91,5,False,False,2.5,2.5
6,Georgia Tech,Georgia Tech,7,10.0,1,4011125042,Virginia,401112504101895808,Virginia,0,1,Bryce Perkins run for 1 yd to the GTech 9,Rush,90,1,False,False,1.0,0.0
9,Georgia Tech,Georgia Tech,7,10.0,1,4011125042,Virginia,401112504101868032,Virginia,0,1,Wayne Taulapapa run for 2 yds to the GTech 49,Rush,49,2,False,False,2.0,0.0


In [42]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = pbp_data[
    (pbp_data.play_type.str.contains('Interception', regex=False))
    | ((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.5 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# away_team Adj Turnovers
away_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == away_team)
    | (adj_turnover_plays.defense == away_team)
]

away_team_ints_def = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_ints_off = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_pds = len(away_team_tos[
   (away_team_tos.play_type == 'Pass Incompletion')
    & (away_team_tos.play_text.str.contains('broken up', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_fum_rec = away_team_tos[((away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.defense == away_team))]
away_team_fum_lost = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.offense == away_team)]

In [43]:
# home_team Adj Turnovers (broken)
home_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == home_team)
    | (adj_turnover_plays.defense == home_team)
]

home_team_ints_def = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_ints_off = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_pds = len(home_team_tos[
   (home_team_tos.play_type == 'Pass Incompletion')
    & (home_team_tos.play_text.str.contains('broken up', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_fum_rec = home_team_tos[((home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.defense == home_team))]
home_team_fum_lost = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.offense == home_team)]

print(away_team)
print("Def INTs:",away_team_ints_def)
print("Off INTs:",away_team_ints_off)
print("Def PDs:",(away_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (away_team_pds + away_team_ints_def))
print("Fum Recovered:",len(away_team_fum_rec))
print("Fum Lost:",len(away_team_fum_lost))
print("Exp Fum:",adj_fum)
away_to = away_team_ints_off + len(away_team_fum_lost)
print("Actual TO:",away_to)
print("Actual TO Margin:",-1 * away_team_ints_off - len(away_team_fum_lost) + away_team_ints_def + len(away_team_fum_rec))
away_team_exp_to = 0.22 * (away_team_pds + away_team_ints_def) + adj_fum
print("Exp TO:",away_team_exp_to)

print("")

print(home_team)
print("Def INTs:",home_team_ints_def)
print("Off INTs:",home_team_ints_off)
print("Def PDs:",(home_team_pds + away_team_ints_def))
print("Exp INTs:",0.22 * (home_team_pds + home_team_ints_def))
print("Fum Recovered:",len(home_team_fum_rec))
print("Fum Lost:",len(home_team_fum_lost))
print("Exp Fum:",adj_fum)
home_to = home_team_ints_off + len(home_team_fum_lost)
print("Actual TO:",home_to)
print("Actual TO Margin:",-1 * home_team_ints_off - len(home_team_fum_lost) + home_team_ints_def + len(home_team_fum_rec))
home_team_exp_to = 0.22 * (home_team_pds + home_team_ints_def) + adj_fum
print("Exp TO:",home_team_exp_to)

Georgia Tech
Def INTs: 0
Off INTs: 1
Def PDs: 0
Exp INTs: 0.0
Fum Recovered: 0
Fum Lost: 0
Exp Fum: 0.0
Actual TO: 1
Actual TO Margin: -1
Exp TO: 0.0

Virginia
Def INTs: 1
Off INTs: 0
Def PDs: 5
Exp INTs: 1.32
Fum Recovered: 0
Fum Lost: 0
Exp Fum: 0.0
Actual TO: 0
Actual TO Margin: 1
Exp TO: 1.32


In [44]:
# home_team_tos

In [45]:
away_team_exp_to_margin = (home_team_exp_to - away_team_exp_to)
print("Exp TO Margin for",away_team,":", away_team_exp_to_margin)
home_team_exp_to_margin = (away_team_exp_to - home_team_exp_to)
print("Exp TO Margin for",home_team,":", home_team_exp_to_margin)

print("TO Luck (pts) for",away_team,":", ((home_to - away_to) - away_team_exp_to_margin) * 5.0)
print("TO Luck (pts) for",home_team,":", ((away_to - home_to) - home_team_exp_to_margin) * 5.0)

Exp TO Margin for Georgia Tech : 1.32
Exp TO Margin for Virginia : -1.32
TO Luck (pts) for Georgia Tech : -11.600000000000001
TO Luck (pts) for Virginia : 11.600000000000001


In [46]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 35, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 60)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 40)")
print(away_team)
away_team_scoring_opps = scoring_opps[
    scoring_opps.offense == away_team
]
print("Total:",len(away_team_scoring_opps))
print("Scored:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]) / len(away_team_scoring_opps))
print("Opps/Drive:",len(away_team_scoring_opps) / len(drives[drives.offense == away_team]))
print("Points/Opp:",(len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'TD']) * 7 + len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'FG'] * 3)) / len(away_team_scoring_opps))

print("")
# home team's scoring opps
print(home_team)
home_team_scoring_opps = scoring_opps[
    scoring_opps.offense == home_team
]
print("Total:",len(home_team_scoring_opps))
print("Scored:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]) / len(home_team_scoring_opps))
print("Opps/Drive:",len(home_team_scoring_opps) / len(drives[drives.offense == home_team]))
print("Points/Opp:",(len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'TD']) * 7 + len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'FG'] * 3)) / len(home_team_scoring_opps))

Scoring Opportunities (IE: Drives inside Opponent's 40)
Georgia Tech
Total: 5
Scored: 4
Opp Efficiency: 0.8
Opps/Drive: 0.5
Points/Opp: 5.6

Virginia
Total: 8
Scored: 6
Opp Efficiency: 0.75
Opps/Drive: 0.8
Points/Opp: 3.75


In [47]:
# Rushing Stats
rush_carries = rush_plays[
    (~rush_plays.play_type.isin(['Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']))
]

# look these up manually
away_team_rushers = ["Jordan Mason","Jerry Howard","Jamious Griffin","James Graham"]
home_team_rushers = ["Bryce Perkins","Wayne Taulapapa","PK Kier","Joe Reed","Lamont Atkins"]

def verify_division(num1, num2):
    if num2 == 0:
        return 0
    else:
        return num1 / num2

print("team level rushing stats")
print(away_team)
print(len(rush_carries[rush_carries.offense == away_team]),"car",
  sum(rush_carries[rush_carries.offense == away_team].yards_gained),"yards",
  len(rush_carries[(rush_carries.offense == away_team) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
  len(rush_plays[(rush_plays.offense == away_team) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
  "(",
  verify_division(sum(rush_carries[(rush_carries.offense == away_team)].yards_gained), len(rush_carries[(rush_carries.offense == away_team)])),"YPC",",",
  verify_division(sum(rush_carries[(rush_carries.offense == away_team)].line_yards), len(rush_carries[(rush_carries.offense == away_team)])),"line YPC",",",
  "SR%:",verify_division(len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))])),",",
  "Opp Rate:",verify_division(len(rush_opps[(rush_opps.offense == away_team)]), len(rush_carries[(rush_carries.offense == away_team)])),",",
  "Hlt/Opp:",verify_division(sum(rush_carries[(rush_carries.offense == away_team)].yards_gained), len(rush_opps[(rush_opps.offense == away_team)])),",",
  "Stuff%:",verify_division(len(stuffed_plays[stuffed_plays.offense == away_team]), len(pbp_data[pbp_data.offense == away_team])),
  ")")
print(home_team)
print(len(rush_carries[rush_carries.offense == home_team]),"car",
  sum(rush_carries[rush_carries.offense == home_team].yards_gained),"yards",
  len(rush_carries[(rush_carries.offense == home_team) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
  len(rush_plays[(rush_plays.offense == home_team) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
  "(",
  verify_division(sum(rush_carries[(rush_carries.offense == home_team)].yards_gained), len(rush_carries[(rush_carries.offense == home_team)])),"YPC",",",
  verify_division(sum(rush_carries[(rush_carries.offense == home_team)].line_yards), len(rush_carries[(rush_carries.offense == home_team)])),"line YPC",",",
  "SR%:",verify_division(len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))])),",",
  "Opp Rate:",verify_division(len(rush_opps[(rush_opps.offense == home_team)]), len(rush_carries[(rush_carries.offense == home_team)])),",",
  "Hlt/Opp:",verify_division(sum(rush_carries[(rush_carries.offense == home_team)].yards_gained), len(rush_opps[(rush_opps.offense == home_team)])),",",
  "Stuff%:",verify_division(len(stuffed_plays[stuffed_plays.offense == home_team]), len(pbp_data[pbp_data.offense == home_team])),
  ")")
    
print("")
print(away_team,"Rushing Stats")
for qb in away_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_type.isin(rush_types))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")

print("")

print(home_team,"Rushing Stats")
for qb in home_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]), len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_type.isin(rush_types))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")


team level rushing stats
Georgia Tech
32 car 158 yards 2 TDs 0 Fum ( 4.9375 YPC , 0.8046875 line YPC , SR%: 0.34375 , Opp Rate: 0.34375 , Hlt/Opp: 14.363636363636363 , Stuff%: 0.09259259259259259 )
Virginia
35 car 165 yards 3 TDs 0 Fum ( 4.714285714285714 YPC , 1.4071428571428573 line YPC , SR%: 0.43243243243243246 , Opp Rate: 0.2571428571428571 , Hlt/Opp: 18.333333333333332 , Stuff%: 0.0547945205479452 )

Georgia Tech Rushing Stats
Jordan Mason: 19 car 109 yards 1 TDs 0 Fum ( 5.7368421052631575 YPC , 0.35526315789473684 line YPC , SR%: 0.3684210526315789 , Opp Rate: 0.3684210526315789 , Hlt/Opp: 15.571428571428571 )
Jerry Howard: 1 car 2 yards 0 TDs 0 Fum ( 2.0 YPC , 2.0 line YPC , SR%: 0.0 , Opp Rate: 0.0 , Hlt/Opp: 0 )
Jamious Griffin: 3 car 7 yards 0 TDs 0 Fum ( 2.3333333333333335 YPC , 1.6666666666666667 line YPC , SR%: 0.3333333333333333 , Opp Rate: 0.0 , Hlt/Opp: 0 )
James Graham: 8 car 42 yards 1 TDs 0 Fum ( 5.25 YPC , 1.8125 line YPC , SR%: 0.375 , Opp Rate: 0.5 , Hlt/Opp: 10.

In [48]:
# Passing Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

pass_completions = pass_attempts[
    (pass_attempts.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
]

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(pbp_data[(pbp_data.offense == team) & (pbp_data.play_type.str.contains("Sack"))]),"Sck")

print("")

# look these up manually
away_team_qbs = ["James Graham"]
home_team_qbs = ["Bryce Perkins"]

print(away_team,"Passing Stats")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")
print("")
print(home_team,"Passing Stats")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")

Georgia Tech: 15 / 22 233 yards 2 TDs 1 INTs 0 Sck
Virginia: 24 / 36 267 yards 1 TDs 0 INTs 2 Sck

Georgia Tech Passing Stats
James Graham: 15 / 22 233 yards 2 TDs 1 INTs

Virginia Passing Stats
Bryce Perkins: 24 / 36 267 yards 1 TDs 0 INTs


In [49]:
# Passing Stats on Std downs
pass_attempts = standard_downs[
    (standard_downs.play_type == 'Pass Reception')
    | (standard_downs.play_type == 'Passing Touchdown')
    | (standard_downs.play_type == 'Pass Incompletion')
    | (standard_downs.play_type.str.contains('Interception'))
]

pass_completions = standard_downs[
    (standard_downs.play_type == 'Pass Reception')
    | (standard_downs.play_type == 'Passing Touchdown')
]

print("Std Downs")

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(standard_downs[(standard_downs.offense == team) & (standard_downs.play_type.str.contains("Sack"))]),"Sck")

print("")

print(away_team,"Passing Stats on Std Downs")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))]),"Sck")
print("")
print(home_team,"Passing Stats on Std Downs")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(standard_downs[(standard_downs.play_text.str.contains(qb)) & (standard_downs.play_type.str.contains("Sack"))]),"Sck")

Std Downs
Georgia Tech: 8 / 10 137 yards 1 TDs 0 INTs 0 Sck
Virginia: 20 / 27 235 yards 1 TDs 0 INTs 0 Sck

Georgia Tech Passing Stats on Std Downs
James Graham: 8 / 10 137 yards 1 TDs 0 INTs 0 Sck

Virginia Passing Stats on Std Downs
Bryce Perkins: 20 / 27 235 yards 1 TDs 0 INTs 0 Sck


In [50]:
# Passing Stats on Pass. downs
pass_attempts = passing_downs[
    (passing_downs.play_type == 'Pass Reception')
    | (passing_downs.play_type == 'Passing Touchdown')
    | (passing_downs.play_type == 'Pass Incompletion')
    | (passing_downs.play_type.str.contains('Interception'))
]

pass_completions = passing_downs[
    (passing_downs.play_type == 'Pass Reception')
    | (passing_downs.play_type == 'Passing Touchdown')
]


print("Pass Downs")

for team in [away_team, home_team]:
    print(team + ":", 
      len(pass_completions[(pass_completions.offense == team)]),
      "/",len(pass_attempts[(pass_attempts.offense == team)]),
      sum(pass_completions[(pass_completions.offense == team)].yards_gained),"yards",
      len(pass_completions[(pass_completions.offense == team) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
      len(pass_attempts[(pass_attempts.offense == team) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
      len(passing_downs[(passing_downs.offense == team) & (passing_downs.play_type.str.contains("Sack"))]),"Sck")

print("")

print(away_team,"Passing Stats on Passing Downs")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
          len(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))]),"Sck")
print("")
print(home_team,"Passing Stats on Passing Downs")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs",
         len(passing_downs[(passing_downs.play_text.str.contains(qb)) & (passing_downs.play_type.str.contains("Sack"))]),"Sck")

Pass Downs
Georgia Tech: 7 / 12 96 yards 1 TDs 1 INTs 0 Sck
Virginia: 4 / 9 32 yards 0 TDs 0 INTs 2 Sck

Georgia Tech Passing Stats on Passing Downs
James Graham: 7 / 12 96 yards 1 TDs 1 INTs 0 Sck

Virginia Passing Stats on Passing Downs
Bryce Perkins: 4 / 9 32 yards 0 TDs 0 INTs 2 Sck


In [51]:
# Recieving Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

pass_completions = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Passing Touchdown')
]

print("Receiving  Stats")
# look these up manually
away_team_wrs = ["Adonicas Sanders","Ahmarean Brown","Tobias Oliver","Dylan Deveney","Tyler Davis","Malachi Carter"]
home_team_wrs = ["Terrell Jana","Billy Kemp IV","Tavares Kelly Jr.","Joe Reed","Tanner Cowley","Wayne Taulapapa"]

print(away_team)
for wr in away_team_wrs:
    print(f"{wr}: {len(pass_attempts[pass_attempts.play_text.str.contains(wr)])} tgt, {len(pass_completions[pass_completions.play_text.str.contains(wr)])} catch, {sum(pass_completions[pass_completions.play_text.str.contains(wr)].yards_gained)} yds, SR%: {verify_division(len(pass_attempts[(pass_attempts.play_text.str.contains(wr) & (pass_attempts.play_successful == True))]),len(pass_attempts[pass_attempts.play_text.str.contains(wr)]))}")

print("")

print(home_team)   
for wr in home_team_wrs:
    print(f"{wr}: {len(pass_attempts[pass_attempts.play_text.str.contains(wr)])} tgt, {len(pass_completions[pass_completions.play_text.str.contains(wr)])} catch, {sum(pass_completions[pass_completions.play_text.str.contains(wr)].yards_gained)} yds, SR%: {verify_division(len(pass_attempts[(pass_attempts.play_text.str.contains(wr) & (pass_attempts.play_successful == True))]),len(pass_attempts[pass_attempts.play_text.str.contains(wr)]))}")

Receiving  Stats
Georgia Tech
Adonicas Sanders: 5 tgt, 5 catch, 25 yds, SR%: 0.6
Ahmarean Brown: 3 tgt, 1 catch, 59 yds, SR%: 0.3333333333333333
Tobias Oliver: 4 tgt, 4 catch, 57 yds, SR%: 0.75
Dylan Deveney: 1 tgt, 1 catch, 13 yds, SR%: 1.0
Tyler Davis: 3 tgt, 3 catch, 54 yds, SR%: 1.0
Malachi Carter: 3 tgt, 1 catch, 25 yds, SR%: 0.3333333333333333

Virginia
Terrell Jana: 12 tgt, 9 catch, 117 yds, SR%: 0.5833333333333334
Billy Kemp IV: 4 tgt, 4 catch, 51 yds, SR%: 0.75
Tavares Kelly Jr.: 2 tgt, 2 catch, 46 yds, SR%: 1.0
Joe Reed: 8 tgt, 5 catch, 27 yds, SR%: 0.375
Tanner Cowley: 2 tgt, 2 catch, 23 yds, SR%: 0.5
Wayne Taulapapa: 3 tgt, 2 catch, 3 yds, SR%: 0.0


In [52]:
# Using 2013's IsoPPP results from BillC's Study Hall
ep_df = pd.read_csv('results/2013-isoppp-values.csv')
ep_df.set_index('yardline', inplace=True)
ep = ep_df.to_dict()

def retrieve_ep(yardline):
    return ep['ep'][yardline]

def retrieve_ep_diff(start_yardline, yards_gained):
    return retrieve_ep(min(max(start_yardline + yards_gained, 1), 100)) - retrieve_ep(start_yardline)


pbp_data.loc[
    pbp_data.offense == away_team, ['yard_line']
] = 100 - pbp_data.yard_line

pbp_data['ppp'] = np.vectorize(retrieve_ep_diff)(pbp_data.yard_line, pbp_data.yards_gained)


standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]


print(away_team)
print('PPP:',sum(pbp_data[(pbp_data.offense == away_team)].ppp) / len(pbp_data[(pbp_data.offense == away_team)]))
print('Passing PPP:',sum(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))].ppp) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(pass_types))]))
print('Rushing PPP:',sum(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))].ppp) / len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_type.isin(rush_types))]))
print('PPP on Std Downs:',sum(standard_downs[(standard_downs.offense == away_team)].ppp) / len(standard_downs[(standard_downs.offense == away_team)]))
print('PPP on Pass Downs:',sum(passing_downs[(passing_downs.offense == away_team)].ppp) / len(passing_downs[(passing_downs.offense == away_team)]))

print("")

print(home_team)
print('PPP:',sum(pbp_data[(pbp_data.offense == home_team)].ppp) / len(pbp_data[(pbp_data.offense == home_team)]))
print('Passing PPP:',sum(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))].ppp) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(pass_types))]))
print('Rushing PPP:',sum(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))].ppp) / len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_type.isin(rush_types))]))
print('PPP on Std Downs:',sum(standard_downs[(standard_downs.offense == home_team)].ppp) / len(standard_downs[(standard_downs.offense == home_team)]))
print('PPP on Pass Downs:',sum(passing_downs[(passing_downs.offense == home_team)].ppp) / len(passing_downs[(passing_downs.offense == home_team)]))


Georgia Tech
PPP: 0.6927777777777779
Passing PPP: 1.0031818181818182
Rushing PPP: 0.47937499999999983
PPP on Std Downs: 0.7094285714285716
PPP on Pass Downs: 0.6621052631578946

Virginia
PPP: 0.5542465753424658
Passing PPP: 0.7138888888888889
Rushing PPP: 0.39891891891891884
PPP on Std Downs: 0.6277192982456139
PPP on Pass Downs: 0.2925000000000001
